In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from skimage.transform import resize
from sklearn.model_selection import train_test_split
import cv2

from tqdm import tqdm
import os
import math

import keras
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPool2D, GlobalMaxPool2D
from keras.models import Model

from keras.preprocessing import image
from keras.utils import to_categorical

### Data Loading and Cleaning

In [5]:
# Open the training set:
file = open(r'data\ucfTrainTestlist\trainlist01.txt', 'r')
temp = file.read()
videos = temp.split('\n')

# Creating a dataframe:
train = pd.DataFrame()
train['video_name'] = videos

# Dropping the last row:
train = train[:-1]


# Opening the test set:
file_test = open(r'data\ucfTrainTestlist\testlist01.txt', 'r')
temp = file_test.read()
videos_test = temp.split("\n")

test = pd.DataFrame()
test['video_name'] = videos_test

test = test[:-1]

In [6]:
# Expanding the video name:
train['class'] = train['video_name'].str.split(" ").apply(lambda x: x[1])

train['class_name'] = train['video_name'].str.split("/").apply(lambda x: x[0])

train['video_name'] = train['video_name'].str.split("/").apply(lambda x: x[1].split(" ")[0])


# Expanding test set:

test['class_name'] = test['video_name'].str.split("/").apply(lambda x: x[0])

test['video_name'] = test['video_name'].str.split("/").apply(lambda x: x[1])



In [9]:
# Storing traning videos:
for i in tqdm(range(train.shape[0])):
    count = 0
    
    video_name = train['video_name'][i]
    video_class = train['class_name'][i]

    path = "data/Videos/{}/{}".format(video_class, video_name)

    video = cv2.VideoCapture(path)
    frame_rate = video.get(7) # O identificador 5 é o FPS.
   
    while video.isOpened():

        frameId = video.get(1) # Identificador 1 é o frame ID.
        ret, frame = video.read()

        if ret != True:
            break
    

        if frameId % math.floor(frame_rate) == 0:
            filename = 'data/ucfTrainTestlist/train_1/' + video_name + '_frame{}.jpg'.format(count)
            cv2.imwrite(filename, frame)
            count+=1

    
    video.release()


100%|██████████| 9537/9537 [24:45<00:00,  6.42it/s]  


In [10]:
# Armazenando todas as imagens em uma variável

imagens = os.listdir("data/ucfTrainTestlist/train_1")
train_labels  =[i.split("_")[1] for i in imagens]


train_df = pd.DataFrame()
train_df['imagens'] = imagens
train_df['labels'] = train_labels

train_df.to_csv("data/train_new.csv", header=True, index=False)

### Preprocessing step

In [12]:
train_df = pd.read_csv("data/train_new.csv")
train_df

,imagens,labels
0,v_ApplyEyeMakeup_g08_c01.avi_frame0.jpg,ApplyEyeMakeup
1,v_ApplyEyeMakeup_g08_c02.avi_frame0.jpg,ApplyEyeMakeup
2,v_ApplyEyeMakeup_g08_c03.avi_frame0.jpg,ApplyEyeMakeup
3,v_ApplyEyeMakeup_g08_c04.avi_frame0.jpg,ApplyEyeMakeup
4,v_ApplyEyeMakeup_g08_c05.avi_frame0.jpg,ApplyEyeMakeup
...,...,...
9532,v_YoYo_g25_c01.avi_frame0.jpg,YoYo
9533,v_YoYo_g25_c02.avi_frame0.jpg,YoYo
9534,v_YoYo_g25_c03.avi_frame0.jpg,YoYo
9535,v_YoYo_g25_c04.avi_frame0.jpg,YoYo


In [13]:
train_image = []

for i in tqdm(range(train_df.shape[0])):

    path = "data/ucfTrainTestlist/train_1/" + train_df["imagens"][i]
    img = image.load_img(path, target_size=(224, 224, 3))

    img = image.img_to_array(img)

    # Normalizando os pixels das imagens:
    img = img/255

    train_image.append(img)

100%|██████████| 9537/9537 [01:00<00:00, 158.13it/s]


In [14]:
X = np.array(train_image)


In [18]:
X.shape

(9537, 224, 224, 3)

In [29]:
y = train_df["labels"]

# Splitting data:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [30]:
# Transformando os labels em variáveis dummy:
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

### Model'S Architecture

In [20]:
# Inicializando o modelo pre-treinado:
base_model = VGG16(weights='imagenet', include_top=False) # inlcude_top define se irá manter a ultima camada ou não.

58889256/58889256 [==============================] - 8s 0us/step


In [ ]:
# Congelando as camadas de treinamento
#for layer in base_model.layers:
#    layer.trainable = False

#classifier1 = base_model.output#head mode
#classifier1 = Flatten()(classifier1)#adding layer of flatten
#classifier1 = Dense(units=256, activation='relu')(classifier1)
#classifier1 = Dropout(0.6)(classifier1)
#classifier1 = Dense(units=40, activation='softmax')(classifier1)

#model = Model(inputs = base_model.input , outputs = classifier1)
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'

In [32]:
print(f"Train shape: {X_train.shape}")
print(f"Test shape: {X_test.shape}")

Train shape: (7629, 224, 224, 3)
Test shape: (1908, 224, 224, 3)


In [33]:
X_train = base_model.predict(X_train)
X_test = base_model.predict(X_test)

print(f"Train shape: {X_train.shape}")
print(f"Test shape: {X_test.shape}")

60/60 [==============================] - 219s 4s/step
Train shape: (7629, 7, 7, 512)
Test shape: (1908, 7, 7, 512)


In [35]:
# reshaping the training as well as validation frames in single dimension
X_train = X_train.reshape(7629, 7*7*512)
X_test = X_test.reshape(1908, 7*7*512)

In [36]:
def fully_connected_layer():
    model = Sequential()
    model.add(Dense(1024, activation='relu', input_shape=(7629, )))
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(101, activation='softmax'))
    return model

In [37]:
model = fully_connected_layer()

In [38]:
from keras.callbacks import ModelCheckpoint

mcp_save = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [39]:
# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [40]:
# training the model
model.fit(X_train, y_train, epochs=200, validation_data=(X_test, y_test), callbacks=[mcp_save], batch_size=128)

Epoch 1/200
60/60 [==============================] - ETA: 0s - loss: 4.8453 - accuracy: 0.0102

c:\Users\caios\Documents\DataScience\ComputerVision\projects\ActionRecognition\.venv\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


60/60 [==============================] - 28s 429ms/step - loss: 4.8453 - accuracy: 0.0102 - val_loss: 4.6119 - val_accuracy: 0.0105
Epoch 2/200
60/60 [==============================] - 25s 424ms/step - loss: 4.6128 - accuracy: 0.0115 - val_loss: 4.6034 - val_accuracy: 0.0362
Epoch 3/200
60/60 [==============================] - 26s 439ms/step - loss: 4.5830 - accuracy: 0.0198 - val_loss: 4.5336 - val_accuracy: 0.0367
Epoch 4/200
60/60 [==============================] - 25s 412ms/step - loss: 4.5350 - accuracy: 0.0287 - val_loss: 4.4908 - val_accuracy: 0.0414
Epoch 5/200
60/60 [==============================] - 24s 406ms/step - loss: 4.4826 - accuracy: 0.0317 - val_loss: 4.3924 - val_accuracy: 0.0655
Epoch 6/200
60/60 [==============================] - 24s 404ms/step - loss: 4.4314 - accuracy: 0.0388 - val_loss: 4.3301 - val_accuracy: 0.0613
Epoch 7/200
60/60 [==============================] - 24s 406ms/step - loss: 4.3650 - accuracy: 0.0380 - val_loss: 4.2367 - val_accuracy: 0.0729
Epoc

KeyboardInterrupt: 